In [2]:
# System imports
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
from time import process_time

# Extra library imports
from PIL import Image
from scipy.optimize import minimize
import scipy.ndimage as ndi
import pandas as pd
import cv2
from skimage.draw import line
from skimage.measure import label, regionprops
import mahotas as mh
from scipy.stats import kurtosis
from scipy.stats import skew
import glob

# **SYMMETRY INDEX**

In [3]:
def symmetry_index(mask_arr):
    # Flip the mask around the y-axis
    flipped_mask = np.fliplr(mask_arr)
    # Calculate the absolute difference between the original and flipped mask
    diff = np.abs(mask_arr - flipped_mask)

    # Calculate the symmetry index
    symmetry_index = np.sum(diff) / np.sum(mask_arr)

    return symmetry_index

# **RATIO TWO LONGEST ORTHO LINES**

In [4]:
def Longest_ortho_lines_ratio(mask_arr,im_arr):
    return 0

# **RATIO NUM_PIX_BUG / NUM_PIX_IM**

In [5]:
def number_of_pixel_ratio(mask_arr, im_arr):
    # Assume mask_arr is your mask array and image is your image array

    # Calculate the number of pixels of the bug
    num_pixels_bug = np.sum(mask_arr > 0)
    # Calculate the number of pixels in the full image
    num_pixels_image = im_arr.size
    # Calculate the ratio
    ratio = num_pixels_bug / num_pixels_image

    return ratio

# **MIN,MAX,MEAN,MEDIAN,STANDARD DEVIATION RGB MASK**

In [6]:
def rgb_mask_features(mask_arr, im_arr):
    # Assuming the bug mask is stored in the variable 'mask_arr' as a numpy array

    # Get the RGB values within the bug mask
    bug_pixels = im_arr[mask_arr == 1]
    # Get the Red, Green, and Blue channels
    red_channel = bug_pixels[:, 0]
    green_channel = bug_pixels[:, 1]
    blue_channel = bug_pixels[:, 2]

    # Calculate the minimum, maximum, and mean values for each channel
    red_min = np.min(red_channel)
    red_max = np.max(red_channel)
    red_mean = np.mean(red_channel)

    green_min = np.min(green_channel)
    green_max = np.max(green_channel)
    green_mean = np.mean(green_channel)

    blue_min = np.min(blue_channel)
    blue_max = np.max(blue_channel)
    blue_mean = np.mean(blue_channel)

    # Calculate median and standard deviation for red channel
    red_median = np.median(red_channel)
    red_std = np.std(red_channel)

    # Calculate median and standard deviation for green channel
    green_median = np.median(green_channel)
    green_std = np.std(green_channel)

    # Calculate median and standard deviation for blue channel
    blue_median = np.median(blue_channel)
    blue_std = np.std(blue_channel)

    red_features = [red_min, red_max, red_mean, red_median, red_std]
    green_features = [green_min, green_max, green_mean, green_median, green_std]
    blue_features = [blue_min, blue_max, blue_mean, blue_median, blue_std]

    return red_features,blue_features,green_features

# **HARALICK TEXTURE FEATURE**

In [7]:
def haralick_features(mask_arr, im_arr):
    # Assuming the bug mask is stored in the variable 'mask_arr' as a numpy array

    # Convert the image to grayscale
    gray_image = mh.colors.rgb2gray(im_arr*mask_arr[:,:,None])
    gray_image = gray_image.astype(np.uint8)

    # Calculate Haralick texture features
    haralick_features = mh.features.haralick(gray_image, return_mean=True)

    return haralick_features

# **ECCENTRICITY FEATURE**

In [8]:
def eccentricity(mask_arr):
    # Calculate the eccentricity
    props = regionprops(mask_arr)
    eccentricity = props[0].eccentricity

    return eccentricity

# **KURTHOSIS,SKEWNESS FEATURE**

In [9]:
def kurthosis_skewness(mask_arr, im_arr):
    # Assuming im_arr is your image array and mask_arr is your mask
    # Apply the mask to the image
    masked_im = im_arr * mask_arr[:,:,None]

    # Flatten the array to 1D for the calculation
    flattened_im = masked_im.flatten()

    # Calculate kurtosis
    kurt = kurtosis(flattened_im)

    # Assuming flattened_im is your flattened image array from the previous step
    # Calculate skewness
    skewness = skew(flattened_im)

    return (kurt,skewness) 

# **Creating all features in a dataset**

In [10]:
def Creating_all_features(mask_arr, im_arr):
    # Calculate the symmetry index
    symmetry = symmetry_index(mask_arr)

    longest_ratio = Longest_ortho_lines_ratio(mask_arr,im_arr)
    # Calculate the number of pixels ratio
    num_pixels_ratio = number_of_pixel_ratio(mask_arr, im_arr)

    # Calculate the RGB features
    rgb_features = rgb_mask_features(mask_arr, im_arr)

    # Calculate the Haralick features
    haralick = haralick_features(mask_arr, im_arr)

    # Calculate the eccentricity
    ecc = eccentricity(mask_arr)

    # Calculate the kurtosis and skewness
    kurt, skewness = kurthosis_skewness(mask_arr, im_arr)

    return (symmetry, longest_ratio, num_pixels_ratio, rgb_features, haralick, ecc, kurt, skewness)

In [11]:
# Get the file paths of the images and masks in the train folder
image_paths = [f'../train/{i}.jpg' for i in range(1, 21)]
mask_paths = [f'../train/masks/binary_{i}.tif' for i in range(1, 21)]


dataframe = pd.DataFrame(columns=['symmetry', 'longest_ratio', 'num_pixels_ratio', 'rgb_features', 'haralick', 'ecc', 'kurt', 'skewness'])

# Iterate over the image and mask paths
for i, (image_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
    print(image_path, mask_path)
    im_arr = np.array(Image.open(image_path))
    mask_arr = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask_arr = np.where(mask_arr > 0, 1, 0)

    # Appeler la fonction Creating_all_features pour obtenir les valeurs
    features = Creating_all_features(mask_arr, im_arr)

    # Create a new DataFrame from the features and append it to df
    dataframe.loc[i] = features


../train/1.jpg ../train/masks/binary_1.tif
../train/2.jpg ../train/masks/binary_2.tif
../train/3.jpg ../train/masks/binary_3.tif
../train/4.jpg ../train/masks/binary_4.tif
../train/5.jpg ../train/masks/binary_5.tif
../train/6.jpg ../train/masks/binary_6.tif
../train/7.jpg ../train/masks/binary_7.tif
../train/8.jpg ../train/masks/binary_8.tif
../train/9.jpg ../train/masks/binary_9.tif
../train/10.jpg ../train/masks/binary_10.tif
../train/11.jpg ../train/masks/binary_11.tif
../train/12.jpg ../train/masks/binary_12.tif
../train/13.jpg ../train/masks/binary_13.tif
../train/14.jpg ../train/masks/binary_14.tif
../train/15.jpg ../train/masks/binary_15.tif
../train/16.jpg ../train/masks/binary_16.tif
../train/17.jpg ../train/masks/binary_17.tif
../train/18.jpg ../train/masks/binary_18.tif
../train/19.jpg ../train/masks/binary_19.tif
../train/20.jpg ../train/masks/binary_20.tif


In [17]:
dataframe

,symmetry,longest_ratio,num_pixels_ratio,rgb_features,haralick,ecc,kurt,skewness
0,0.695058,0,0.002476,"([5, 208, 68.08574489687165, 56.0, 47.95539885...",[ 9.84840515e-01 4.63390039e+00 9.40188055e-...,0.392019,415.810811,19.208069
1,0.833665,0,0.002851,"([2, 248, 63.786497914961615, 55.0, 42.1535075...",[ 9.82586219e-01 5.34701353e+00 9.31199490e-...,0.595904,330.090332,17.303837
2,1.584773,0,0.007364,"([3, 255, 107.17633313530793, 114.0, 58.771358...",[ 9.55824847e-01 1.02604060e+01 9.79688327e-...,0.240295,95.486536,9.429777
3,1.989106,0,0.004396,"([5, 219, 87.58824254258334, 88.0, 46.32258744...",[ 9.73306968e-01 8.63780557e+00 9.55254889e-...,0.735434,155.355812,12.064838
4,1.265982,0,0.003055,"([6, 255, 123.1770030597724, 134.0, 62.3559441...",[ 9.81439860e-01 9.41018266e+00 9.65218503e-...,0.650599,203.554722,13.915533
5,1.989437,0,0.001972,"([7, 201, 100.47562040505902, 109.0, 49.635040...",[ 9.87979333e-01 4.13862074e+00 9.65181940e-...,0.764470,316.071620,17.277534
6,2.000000,0,0.001886,"([11, 238, 91.57896325749209, 87.0, 41.3787496...",[ 9.88476413e-01 3.28801092e+00 9.58551080e-...,0.900116,413.600942,18.821637
7,2.000000,0,0.002485,"([4, 168, 55.60429406701025, 47.0, 35.66732309...",[ 9.84859461e-01 3.07948161e+00 9.37852664e-...,0.875602,520.105458,20.903357
8,1.498709,0,0.002400,"([6, 255, 87.62007338557522, 93.0, 52.73279931...",[ 9.85360485e-01 3.38021673e+00 9.71904929e-...,0.513316,291.352680,16.514611
9,1.363680,0,0.005261,"([4, 255, 82.97670029012441, 80.0, 52.42158090...",[ 9.68334891e-01 4.88504067e+00 9.78182921e-...,0.472818,147.189839,11.632354


In [18]:
dataframe.to_csv('dataframe.csv', index=False)


In [19]:
dataframe_import = pd.read_csv('dataframe.csv')


In [20]:
dataframe_import

,symmetry,longest_ratio,num_pixels_ratio,rgb_features,haralick,ecc,kurt,skewness
0,0.695058,0,0.002476,"([5, 208, 68.08574489687165, 56.0, 47.95539885...",[ 9.84840515e-01 4.63390039e+00 9.40188055e-...,0.392019,415.810811,19.208069
1,0.833665,0,0.002851,"([2, 248, 63.786497914961615, 55.0, 42.1535075...",[ 9.82586219e-01 5.34701353e+00 9.31199490e-...,0.595904,330.090332,17.303837
2,1.584773,0,0.007364,"([3, 255, 107.17633313530793, 114.0, 58.771358...",[ 9.55824847e-01 1.02604060e+01 9.79688327e-...,0.240295,95.486536,9.429777
3,1.989106,0,0.004396,"([5, 219, 87.58824254258334, 88.0, 46.32258744...",[ 9.73306968e-01 8.63780557e+00 9.55254889e-...,0.735434,155.355812,12.064838
4,1.265982,0,0.003055,"([6, 255, 123.1770030597724, 134.0, 62.3559441...",[ 9.81439860e-01 9.41018266e+00 9.65218503e-...,0.650599,203.554722,13.915533
5,1.989437,0,0.001972,"([7, 201, 100.47562040505902, 109.0, 49.635040...",[ 9.87979333e-01 4.13862074e+00 9.65181940e-...,0.764470,316.071620,17.277534
6,2.000000,0,0.001886,"([11, 238, 91.57896325749209, 87.0, 41.3787496...",[ 9.88476413e-01 3.28801092e+00 9.58551080e-...,0.900116,413.600942,18.821637
7,2.000000,0,0.002485,"([4, 168, 55.60429406701025, 47.0, 35.66732309...",[ 9.84859461e-01 3.07948161e+00 9.37852664e-...,0.875602,520.105458,20.903357
8,1.498709,0,0.002400,"([6, 255, 87.62007338557522, 93.0, 52.73279931...",[ 9.85360485e-01 3.38021673e+00 9.71904929e-...,0.513316,291.352680,16.514611
9,1.363680,0,0.005261,"([4, 255, 82.97670029012441, 80.0, 52.42158090...",[ 9.68334891e-01 4.88504067e+00 9.78182921e-...,0.472818,147.189839,11.632354
